# Make / Model Reference Construction

In this notebook, we build a clean and structured reference dataset of vehicle **makes** and **models** to support intelligent matching against real-world listings from Craiglist.

## Challenge

Craiglist listings include a free-text field for vehicle names (e.g. `BMW 328i`, `Mercedes C-Class`, etc.) which are often messy and inconsistent. To cleanly match these to structured makes and models, we need a reliable reference dataset that includes:

- All major **automotive brands** (makes)
- The **commercial names** of models (e.g. `3 Series`, `C-Class`, `Cayenne`)
- Their **known variants and aliases** (e.g. `328i`, `C43 AMG`, `Cayenne GTS`)

We will document the creation of such a reference in this notebook, including:

- Cleaning and standardizing make/model names
- Grouping variants under canonical model families

## External Ressource

We use the dataset provided by the U.S. EPA, published on Opendatasoft:

[All Vehicles Model – public.opendatasoft.com](https://public.opendatasoft.com/explore/dataset/all-vehicles-model/information/)

This file serves as the foundation for matching vehicle listings using fuzzy logic in our Craiglist processing pipeline.

It includes over 45,000 vehicle entries with rich attributes, including the `Make`, `Model`, and a normalized `baseModel` which we use as our **model family name**.

Additionaly, it has a **lot of referentials** (Transmission,Drive,Size...) on every car that we can reuse for our analysis.



### Explore the Opendatasoft's dataset:
[All Vehicles Model – public.opendatasoft.com](https://public.opendatasoft.com/explore/dataset/all-vehicles-model/information/)

In [3]:
import pandas as pd
df = pd.read_csv("../data/all-vehicles-model.csv", sep = ";")
df.head()

,Make,Model,Annual Petroleum Consumption For Fuel Type1,Annual Petroleum Consumption For Fuel Type2,Time to charge at 120V,Time to charge at 240V,City Mpg For Fuel Type1,Unrounded City Mpg For Fuel Type1 (2),City Mpg For Fuel Type2,Unrounded City Mpg For Fuel Type2,...,c240Dscr,charge240b,C240B Dscr,Created On,Modified On,Start-Stop,PHEV City,PHEV Highway,PHEV Combined,baseModel
0,Ford,Bronco 4WD,22.885385,0.0,0.0,0.0,12,0.0,0,0.0,...,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0,Bronco
1,Ford,LTD Wagon,15.658421,0.0,0.0,0.0,17,0.0,0,0.0,...,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0,LTD
2,Chevrolet,S10 Pickup 2WD,12.935217,0.0,0.0,0.0,21,0.0,0,0.0,...,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0,S10 Pickup
3,Dodge,Ram 50 Pickup 2WD,14.875500,0.0,0.0,0.0,18,0.0,0,0.0,...,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0,Ram 50 Pickup
4,Ford,Ranger Pickup 2WD,15.658421,0.0,0.0,0.0,18,0.0,0,0.0,...,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0,Ranger


The CSV has 84 columns,we will not use all of them. Let's create two sub-datasets:

* **`df_ref`**: This will keep all the referential data such as the **Size**, **Cylinders**, or other specifications of a car.
* **`df_fuzzy`**: This will contain the **minimum set of information** required to perform a fuzzy match between the `MakeModel` column of our Craiglist dataset and the actual `Make` / `Model` from this reference dataset.

In [5]:
df_ref = df
df_ref.describe(include='all')

,Make,Model,Annual Petroleum Consumption For Fuel Type1,Annual Petroleum Consumption For Fuel Type2,Time to charge at 120V,Time to charge at 240V,City Mpg For Fuel Type1,Unrounded City Mpg For Fuel Type1 (2),City Mpg For Fuel Type2,Unrounded City Mpg For Fuel Type2,...,c240Dscr,charge240b,C240B Dscr,Created On,Modified On,Start-Stop,PHEV City,PHEV Highway,PHEV Combined,baseModel
count,47523,47523,47523.000000,47523.000000,47523.0,47523.000000,47523.000000,47523.000000,47523.000000,47523.000000,...,141,47523.000000,135,47523,47523,15834,47523.000000,47523.000000,47523.000000,47523
unique,144,5064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5,NaN,7,455,298,2,NaN,NaN,NaN,1451
top,Chevrolet,F150 Pickup 2WD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,standard charger,NaN,80 amp dual charger,2013-01-01,2013-01-01,N,NaN,NaN,NaN,Sierra
freq,4420,236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130,NaN,55,34193,29169,8621,NaN,NaN,NaN,733
mean,NaN,NaN,15.173212,0.190030,0.0,0.178136,19.662079,8.940989,0.896303,0.770897,...,NaN,0.015435,NaN,NaN,NaN,NaN,0.311302,0.313553,0.310923,NaN
std,NaN,NaN,4.488236,0.976448,0.0,1.313176,11.914322,15.857898,6.821359,6.755519,...,NaN,0.321030,NaN,NaN,NaN,NaN,4.035418,3.894750,3.946077,NaN
min,NaN,NaN,0.047081,0.000000,0.0,0.000000,6.000000,0.000000,0.000000,0.000000,...,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
25%,NaN,NaN,12.396250,0.000000,0.0,0.000000,15.000000,0.000000,0.000000,0.000000,...,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
50%,NaN,NaN,14.875500,0.000000,0.0,0.000000,18.000000,0.000000,0.000000,0.000000,...,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
75%,NaN,NaN,17.500588,0.000000,0.0,0.000000,21.000000,17.598600,0.000000,0.000000,...,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN


In [6]:
df_fuzzy = df[['Make', 'Model','baseModel']].drop_duplicates().reset_index(drop= True)
df_fuzzy[['make', 'model', 'basemodel']] = df_fuzzy[['Make', 'Model', 'baseModel']].apply(lambda col: col.str.upper())
df_fuzzy['make'] = df_fuzzy['make'].str.upper().replace(r'[^A-Z0-9]', '', regex=True)
df_fuzzy['make'] = df_fuzzy['make'].replace('MERCEDESBENZ', 'MERCEDES')
df_fuzzy['make'] = df_fuzzy['make'].replace('MCLARENAUTOMOTIVE', 'MCLAREN')

In [7]:
df_fuzzy.head()

,Make,Model,baseModel,make,model,basemodel
0,Ford,Bronco 4WD,Bronco,FORD,BRONCO 4WD,BRONCO
1,Ford,LTD Wagon,LTD,FORD,LTD WAGON,LTD
2,Chevrolet,S10 Pickup 2WD,S10 Pickup,CHEVROLET,S10 PICKUP 2WD,S10 PICKUP
3,Dodge,Ram 50 Pickup 2WD,Ram 50 Pickup,DODGE,RAM 50 PICKUP 2WD,RAM 50 PICKUP
4,Ford,Ranger Pickup 2WD,Ranger,FORD,RANGER PICKUP 2WD,RANGER


The **resulting dataset (`df_fuzzy`)** is structured as follows:

- One row per model
- Each alias is linked to:
  - A `make` (uppercase)
  - A `BaseModel` (canonical model family)
  - A `Model` (variant used in listings)

Now that we have this dictionnary, we can import the **`craiglist_dataset`** and start working on it.

In [10]:
import pandas as pd

df = pd.read_csv("../data/craiglist_listings.csv")

### Manage the car maker information:

In the **`craiglist_dataset`** The information on the make and the model are stored in the same column "`makemodel`".

Let's visualize it:

In [13]:
df['makemodel']

0                           ACURA TSX
1                            ACURA TL
2                            ACURA TL
3                      ford fusion se
4           honda accord 4-door sedan
                     ...             
128014    ford f350 super duty lariat
128015                      bmw m340i
128016               Mercedes s55 amg
128017                        Audi S4
128018                 toyota corolla
Name: makemodel, Length: 128019, dtype: object

Most listings already include the **car make** within the `makemodel` column, although often written with inconsistent formatting such as mixed casing, extra spaces, or hyphens.

To standardize the `makemodel` column, we'll:
* Remove spaces and hyphens using a regex
* Convert all text to **uppercase** for consistency.

In [15]:
df['makemodel'] = df['makemodel'].str.upper().replace(r'[^A-Z0-9]', '', regex=True)
df['makemodel'].head(10)

0                 ACURATSX
1                  ACURATL
2                  ACURATL
3             FORDFUSIONSE
4    HONDAACCORD4DOORSEDAN
5              HONDAACCORD
6              HONDAACCORD
7              HONDAACCORD
8         HONDAACCORDSPORT
9        TOYOTALANDCRUISER
Name: makemodel, dtype: object

Knowing that, we can now create a function `map_make` to affect a *"generic car make"* to each listing:

In [17]:
def map_make(makemodel, makes):
    if not isinstance(makemodel, str):
        return None

    for make in makes:
        if make == "CHEVROLET":
            if "CHEVROLET" in makemodel or "CHEV" in makemodel:
                return make
        if make == "LANDROVER":
            if "LANDROVER" in makemodel or "RANGEROVER" in makemodel:
                return make
        else:
            if make in makemodel:
                return make

    return None

In [18]:
df['make_mapped'] = df['makemodel'].apply(lambda x: map_make(x, df_fuzzy['make']))

missing_rate = round(100*len(df[df['make_mapped'].isna()][['makemodel']].drop_duplicates())/len(df),2)

print("Mapping Summary:")
print(f"→ {missing_rate}% of car models could not be mapped to a known make.")

Mapping Summary:
→ 1.32% of car models could not be mapped to a known make.


### Car Models

For the models, it's a different story...

To better structure our vehicle data, we will define **two levels of model granularity**:

- **`model`**:  
  The *commercial name* most commonly recognized by users and manufacturers.  
  Examples: `C-CLASS`, `5-SERIES`, `Q7`.

- **`specific_model`**:  
  A more detailed identifier, often including information about the **engine**, **drivetrain**, or **trim level**.  
  Examples: `C300 4MATIC`, `528i xDrive`, `Q7 3.0 TFSI`.

---

To simplify fuzzy matching and ensure consistency, we’ll format the `makemodel` values using the same cleaning steps as for the car makes:
- Remove spaces and hyphens using a **regex**,
- Convert all text to **uppercase**.

In [21]:
df_fuzzy['specific_model'] = df_fuzzy['Model'].str.upper().replace(r'[^A-Z0-9]', '', regex=True)
df_fuzzy['model'] = df_fuzzy['baseModel'].str.upper().replace(r'[^A-Z0-9]', '', regex=True)

In the **`Craigslist dataset`**, we already have a clean and standardized column for the **car make** (`make_mapped`).

To improve the quality of our fuzzy matching (especially when identifying car models), we will process the `makemodel` column by **removing the car make** from it. This allows us to isolate the part of the string that represents the actual model or variant.

For example:
- `"FORDESCAPE"` becomes `"ESCAPE"`
- `"DODGERAM1500"` becomes `"RAM1500"`

In [23]:
df['makemodel_withoutmake'] = df.apply(
    lambda row: row['makemodel']
        .replace(row['make_mapped'], '', 1)
        .replace('BENZ', '') #We have a recurrent and non-negligeable issue with MERCEDESBENZ != MERCEDES
        if isinstance(row['makemodel'], str) and isinstance(row['make_mapped'], str)
        else row['makemodel'],
    axis=1
)

**Here some Data Science!**

We define a function to match each listing to known car models using fuzzy logic.

Logic:
- For each row, we use the brand (`make_mapped`) and cleaned model name (`makemodel_withoutmake`).
- First, we try to match it against the most detailed model (`specific_model`) from a reference table.
- If a strong match is found (e.g. similarity ≥ 80), we return both the matched `specific_model` and its corresponding general `model`.
- If no good match, we fall back to matching the broader `model` directly.
- If nothing matches, we return `None` for both.

In [25]:
from rapidfuzz import process, fuzz

def match_model(row, df_fuzzy, model_threshold=80, modelbase_threshold=90):
    make = row['make_mapped']
    model = row['makemodel_withoutmake']

    candidates = df_fuzzy[df_fuzzy['make'] == make]

    if not candidates['specific_model'].isna().all():
        models = candidates['specific_model'].dropna().unique()
        match = process.extractOne(model, models, scorer=fuzz.partial_ratio)
        if match and match[1] >= model_threshold:
            matched_model = match[0]
            matched_modelbase = candidates[candidates['specific_model'] == matched_model]['model'].iloc[0]
            return pd.Series([matched_model, matched_modelbase])

    # Step 2: fallback fuzzy match on 'model'
    if not candidates['model'].isna().all():
        modelbases = candidates['model'].dropna().unique()
        match = process.extractOne(model, modelbases, scorer=fuzz.partial_ratio)
        if match and match[1] >= modelbase_threshold:
            matched_modelbase = match[0]
            return pd.Series([None, matched_modelbase])

    return pd.Series([None, None])


In [26]:
df[['specific_model', 'model']]=df.apply(lambda row: match_model(row, df_fuzzy), axis=1)

In [27]:
df.head()

,url,title,price,location,year,makemodel,condition,cylinders,drive,fuel,...,paint_color,title_status,transmission,type,posted_at,region,make_mapped,makemodel_withoutmake,specific_model,model
0,https://atlanta.craigslist.org/atl/ctd/d/2010-...,2010 Acura TSX Leather! NAV! SunRoof! TINT! $9...,$999,Marietta,2009.0,ACURATSX,like new,4 cylinders,fwd,gas,...,brown,clean,automatic,sedan,2025-06-13,atlanta,ACURA,TSX,TSX,TSX
1,https://atlanta.craigslist.org/atl/ctd/d/2012-...,2012 Acura TL Leather! SunRoof! $1500 DOWN! N...,"$1,500",Marietta,2012.0,ACURATL,good,6 cylinders,fwd,gas,...,blue,clean,automatic,sedan,2025-06-18,atlanta,ACURA,TL,TL4WD,TL
2,https://atlanta.craigslist.org/atl/ctd/d/2012-...,2012 Acura TL Leather! SunRoof! $999 DOWN! NO...,$999,Marietta,2014.0,ACURATL,good,6 cylinders,fwd,gas,...,white,clean,automatic,sedan,2025-06-13,atlanta,ACURA,TL,TL4WD,TL
3,https://atlanta.craigslist.org/atl/ctd/d/2012-...,2012 FORD FUSION SE! CLEAN!! SunRoof! $699 DOW...,$699,Marietta,2007.0,FORDFUSIONSE,like new,4 cylinders,fwd,gas,...,blue,clean,automatic,sedan,2025-06-18,atlanta,FORD,FUSIONSE,FUSION,FUSION
4,https://atlanta.craigslist.org/atl/ctd/d/2012-...,2012 Honda Accord EX-L $889 DN! LEATHER! ROOF!...,$899,Marietta,2012.0,HONDAACCORD4DOORSEDAN,like new,4 cylinders,fwd,gas,...,silver,clean,automatic,sedan,2025-07-03,atlanta,HONDA,ACCORD4DOORSEDAN,ACCORD,ACCORD


Unless a very specific use case arises, we will mainly rely on the standardized columns:
- `make`: the cleaned car brand,
- `model`: the general, commercial model name.

These fields are now consistent across listings and ready for analysis.

### Add other valuable information

Now that we have the specific_model for each listing, we can retrieve model by model information from the `data_ref` dataset.

Here is the list of the **columns** available:

In [30]:
df_ref.columns

Index(['Make', 'Model', 'Annual Petroleum Consumption For Fuel Type1',
       'Annual Petroleum Consumption For Fuel Type2', 'Time to charge at 120V',
       'Time to charge at 240V', 'City Mpg For Fuel Type1',
       'Unrounded City Mpg For Fuel Type1 (2)', 'City Mpg For Fuel Type2',
       'Unrounded City Mpg For Fuel Type2', 'City gasoline consumption',
       'City electricity consumption', 'EPA city utility factor',
       'Co2 Fuel Type1', 'Co2 Fuel Type2', 'Co2  Tailpipe For Fuel Type2',
       'Co2  Tailpipe For Fuel Type1', 'Combined Mpg For Fuel Type1',
       'Unrounded Combined Mpg For Fuel Type1', 'Combined Mpg For Fuel Type2',
       'Unrounded Combined Mpg For Fuel Type2',
       'Combined electricity consumption', 'Combined gasoline consumption',
       'EPA combined utility factor', 'Cylinders', 'Engine displacement',
       'Drive', 'EPA model type index', 'Engine descriptor',
       'EPA Fuel Economy Score', 'Annual Fuel Cost For Fuel Type1',
       'Annual Fuel Cost

I will determine, on my knowledge of the automativ, the relevant information.

Columns selected:
* Engine displacement
* Fuel Type 1
* Transmission
* Vehicule Size Class
* Unrounded combined MPG for fuelType1
* Annual Fuel Cost For Fuel Type1
* GHG Score
* Unrounded Highway for FuelType1
* ATV Type
* Electric motor
* Start-Stop

We can **map** eveyrthing by using the `specific_model` and the `Year` each listing:

In [32]:
df_ref['specific_model'] = df_ref['Model'].str.upper().replace(r'[^A-Z0-9]', '', regex=True)

cols_selected = [
    'Engine displacement', 'Fuel Type1', 'Transmission',
    'Vehicle Size Class', 'Unrounded Combined Mpg For Fuel Type1',
    'Annual Fuel Cost For Fuel Type1', 'GHG Score',
    'Unrounded Highway Mpg For Fuel Type1', 'ATV Type',
    'Electric motor', 'Start-Stop'
]


df = df.merge(
    df_ref[["specific_model", "Year"] + cols_selected],
    left_on=["specific_model", "year"],
    right_on=["specific_model", "Year"],
    how="left"
)

### Final output and export

We can now **export this enriched dataset**, which includes valuable structured information for each car listing — cleaned, normalized, and ready for downstream tasks such as:
- Price prediction,
- Market analysis,
- Brand/model distribution,
- and more.

In [35]:
df.to_csv("../data/all-vehicles-model__enriched.csv", index=False)
df.head()

,url,title,price,location,year,makemodel,condition,cylinders,drive,fuel,...,Fuel Type1,Transmission,Vehicle Size Class,Unrounded Combined Mpg For Fuel Type1,Annual Fuel Cost For Fuel Type1,GHG Score,Unrounded Highway Mpg For Fuel Type1,ATV Type,Electric motor,Start-Stop
0,https://atlanta.craigslist.org/atl/ctd/d/2010-...,2010 Acura TSX Leather! NAV! SunRoof! TINT! $9...,$999,Marietta,2009.0,ACURATSX,like new,4 cylinders,fwd,gas,...,Premium Gasoline,Automatic (S5),Compact Cars,0.0000,2700.0,NaN,0.0000,NaN,NaN,NaN
1,https://atlanta.craigslist.org/atl/ctd/d/2010-...,2010 Acura TSX Leather! NAV! SunRoof! TINT! $9...,$999,Marietta,2009.0,ACURATSX,like new,4 cylinders,fwd,gas,...,Premium Gasoline,Manual 6-spd,Compact Cars,0.0000,2950.0,NaN,0.0000,NaN,NaN,NaN
2,https://atlanta.craigslist.org/atl/ctd/d/2012-...,2012 Acura TL Leather! SunRoof! $1500 DOWN! N...,"$1,500",Marietta,2012.0,ACURATL,good,6 cylinders,fwd,gas,...,Premium Gasoline,Automatic (S6),Midsize Cars,21.1028,3200.0,NaN,25.8687,NaN,NaN,NaN
3,https://atlanta.craigslist.org/atl/ctd/d/2012-...,2012 Acura TL Leather! SunRoof! $1500 DOWN! N...,"$1,500",Marietta,2012.0,ACURATL,good,6 cylinders,fwd,gas,...,Premium Gasoline,Manual 6-spd,Midsize Cars,20.0986,3350.0,NaN,24.4967,NaN,NaN,NaN
4,https://atlanta.craigslist.org/atl/ctd/d/2012-...,2012 Acura TL Leather! SunRoof! $999 DOWN! NO...,$999,Marietta,2014.0,ACURATL,good,6 cylinders,fwd,gas,...,Premium Gasoline,Automatic (S6),Midsize Cars,21.1028,3200.0,5.0,25.8687,NaN,NaN,N
